## Indexing

In [ ]:
#langchain and openai laibrarys
!pip install --upgrade langchain langchain-openai langchain-core langsmith langchain_community
#Vector Store
!pip install Chromadb
#for the embading model
!pip install langchain_google_genai
#pdf
!pip install pypdf

In [ ]:
!pip install pypdf

In [ ]:
import os
from getpass import getpass

api_key = getpass("Enter your OpenAI API key: ")
os.environ["GEMINI_API_KEY"] = api_key

Enter your OpenAI API key: ··········


### Prepare a loader object for the file

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
loader_pdf =PyPDFLoader("/content/Introduction_to_Tableau.pdf")

In [ ]:
docs_list= loader_pdf.load()
len(docs_list)

49

In [ ]:
docs_list[0].metadata

{'producer': 'Microsoft® Word for Microsoft 365',
 'creator': 'Microsoft® Word for Microsoft 365',
 'creationdate': '2024-10-11T11:01:10+03:00',
 'author': 'python-docx',
 'moddate': '2024-10-11T11:01:10+03:00',
 'source': '/content/Introduction_to_Tableau.pdf',
 'total_pages': 49,
 'page': 0,
 'page_label': '1'}

In [ ]:
import copy
#make a deepcopy to less docs_list as his form when we make changes
docs_list_copy=copy.deepcopy(docs_list)

In [ ]:
#the MHTS class accepts a single string as an argument so we need to concatenate the contents of the docs_list into a string
string_list_contcat="\n".join([doc.page_content for doc in docs_list_copy])
len(string_list_contcat)

72734

### Spliting the pdf whit a **markdownheadertextsplitter**

In [ ]:
from langchain_text_splitters.character import CharacterTextSplitter
from langchain_text_splitters.markdown import MarkdownHeaderTextSplitter

In [ ]:
md_splitter =MarkdownHeaderTextSplitter(headers_to_split_on=[('#','Section Title'),
                                                              ('##',"Course Title")])

In [ ]:
docs_list_md_split=md_splitter.split_text(string_list_contcat)

In [ ]:
len(docs_list_md_split)

25

In [ ]:
docs_list_md_split[0].metadata

{'Section Title': 'Introduction to Tableau',
 'Course Title': 'Welcome to Tableau'}

now we have the transcript split into documents based on heders and sections

In [ ]:
docs_list_md_split[0].page_content

"Hi, everyone.\nI'm Ned and I'll be your instructor for this\ncourse.\nTableau is an invaluable tool.\nOne needs to learn on their journey to become a\nsuccessful business intelligence analyst or\ndata scientist.\nThe art of these professions is storytelling\nusing data to tell stories and convince top\nmanagement of the right course of action.\nBy completing this part of the program, you\nwill know how to create charts and dashboards\nin tableaux.\nThis is an essential step on your way to a data\nscientist role."

we can see that there is some mistacs in the content of the sections ( I'm Ned and a , is needed after Ned and for this\ncourse should be one sentence and alot more)
so we need to correcte this

### Create a chain to correct the the errors

The approach would be to create a simple LCEL chain consisting of a chat prompt template (where we'll pass the lecture scripts one at a time), a chat model, and a sting output parser. We'll then invoke the chain to pass all lecture texts in parallel.

In [ ]:
string_list_split=[doc.page_content for doc in docs_list_md_split]
len(string_list_split),string_list_split[0]

(25,
 "Hi, everyone.\nI'm Ned and I'll be your instructor for this\ncourse.\nTableau is an invaluable tool.\nOne needs to learn on their journey to become a\nsuccessful business intelligence analyst or\ndata scientist.\nThe art of these professions is storytelling\nusing data to tell stories and convince top\nmanagement of the right course of action.\nBy completing this part of the program, you\nwill know how to create charts and dashboards\nin tableaux.\nThis is an essential step on your way to a data\nscientist role.")

In [ ]:
from langchain_core.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate,PromptTemplate
from langchain_core.messages import SystemMessage

In [ ]:
PROMPT_FORMATTING_S = """ Improve the following Tableau lecture transcript by:
- Splitting the text into meaningful paragraphs
- Correcting any misplaced punctuation
- Fixing mistranscribed words (e.g., changing 'tableaux' to 'Tableau')"
- Give just the improved trascript without anythine more
"""

PROMPT_TEMPLATE_FORMATTING_H = """ This is the transcript:
{lecture_transcript}
"""

In [ ]:
prompt_formatting_s=SystemMessagePromptTemplate.from_template(PROMPT_FORMATTING_S)
prompt_template_formatting_h=HumanMessagePromptTemplate.from_template(PROMPT_TEMPLATE_FORMATTING_H)


In [ ]:
chat_propt_template_formatting=ChatPromptTemplate.from_messages([prompt_formatting_s,prompt_template_formatting_h])

In [ ]:
chat_propt_template_formatting

ChatPromptTemplate(input_variables=['lecture_transcript'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template=' Improve the following Tableau lecture transcript by:\n- Splitting the text into meaningful paragraphs\n- Correcting any misplaced punctuation\n- Fixing mistranscribed words (e.g., changing \'tableaux\' to \'Tableau\')"\n- Give just the improved trascript without anythine more\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['lecture_transcript'], input_types={}, partial_variables={}, template=' This is the transcript:\n{lecture_transcript}\n'), additional_kwargs={})])

In [ ]:
from langchain_openai.chat_models import ChatOpenAI

chat=ChatOpenAI(model="models/gemini-2.5-flash-lite",
                api_key=os.environ["GEMINI_API_KEY"],
              base_url="https://generativelanguage.googleapis.com/v1beta/openai/v1",
               temperature=0,


            )

In [ ]:
from langchain_core.output_parsers.string import StrOutputParser

In [ ]:
str_output_parser=StrOutputParser()

In [ ]:
chain_formatting=chat_propt_template_formatting|chat|str_output_parser

In [ ]:
chain_test1=chat_propt_template_formatting|chat

In [ ]:
chain_test1.invoke({'lecture_transcript':string_list_split[0]})

AIMessage(content="Hi, everyone. I'm Ned, and I'll be your instructor for this course.\n\nTableau is an invaluable tool one needs to learn on their journey to become a successful business intelligence analyst or data scientist. The art of these professions is storytelling, using data to tell stories and convince top management of the right course of action.\n\nBy completing this part of the program, you will know how to create charts and dashboards in Tableau. This is an essential step on your way to a data scientist role.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 104, 'prompt_tokens': 184, 'total_tokens': 288, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'models/gemini-2.5-flash-lite', 'system_fingerprint': None, 'id': 'RI4faemyO_m01e8PhJCqmAw', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--9c0d3470-7f66-4494-ac41-060ae8f9c069-0', usage_metadata={'input_tokens

In [ ]:
chain_formatting.invoke({'lecture_transcript':string_list_split[0]})

"Hi, everyone. I'm Ned, and I'll be your instructor for this course.\n\nTableau is an invaluable tool one needs to learn on their journey to become a successful business intelligence analyst or data scientist. The art of these professions is storytelling, using data to tell stories and convince top management of the right course of action.\n\nBy completing this part of the program, you will know how to create charts and dashboards in Tableau. This is an essential step on your way to a data scientist role."

In [ ]:
string_list_split[0]

"Hi, everyone.\nI'm Ned and I'll be your instructor for this\ncourse.\nTableau is an invaluable tool.\nOne needs to learn on their journey to become a\nsuccessful business intelligence analyst or\ndata scientist.\nThe art of these professions is storytelling\nusing data to tell stories and convince top\nmanagement of the right course of action.\nBy completing this part of the program, you\nwill know how to create charts and dashboards\nin tableaux.\nThis is an essential step on your way to a data\nscientist role."

In [ ]:
def correcte_transcript(text):
  return chain_formatting.invoke({'lecture_transcript':text})



In [ ]:
for i in range(len(string_list_split)):
  docs_list_md_split[i].page_content=correcte_transcript(string_list_split[i])

this take time as we have 25 requeste to send and response so it will be better if we do this on paralle

---



```
result=chain_formatting.batch([{'lecture_transcript':text } for text in string_list_split])
 for i in range(len(result)):
    docs_list_md_split[i].page_content=result[i]
```



Split the transcript with TokenTextSplitter

In [ ]:
from langchain_text_splitters import TokenTextSplitter

In [ ]:
token_spliter=TokenTextSplitter(encoding_name='cl100k_base',chunk_size=500,chunk_overlap=50)

In [ ]:
docs_list_tokens_split=token_spliter.split_documents(docs_list_md_split)

In [ ]:
len(docs_list_tokens_split),len(docs_list_md_split)

(44, 25)

In [ ]:
len(docs_list_md_split[1].page_content),len(docs_list_tokens_split[1].page_content)

(3429, 2657)

### Embedding - Vector Store

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
embedding = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001",google_api_key=os.environ['GEMINI_API_KEY'])

In [ ]:
from langchain_community.vectorstores import Chroma

In [ ]:
vectorstore=Chroma.from_documents(documents=docs_list_tokens_split,embedding=embedding,persist_directory='/content/intro-to-tableau')

In [ ]:
len((vectorstore.get())['ids'])

44

In [ ]:
vectorstore.get().keys()

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'included', 'data', 'metadatas'])

## Retrievel

In [ ]:
retriever=vectorstore.as_retriever(search_type='mmr',search_kwargs={'k':2,'lambda_mult':0.7})

In [ ]:
retriever

VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7a6e385c7b00>, search_type='mmr', search_kwargs={'k': 2, 'lambda_mult': 0.7})

## Generation

Greate Prompts and prompt templates for the Q&A chatbot chain

In [ ]:
PROMPT_CREATING_QUESTION = '''Lecture: {question_lecture}
Title: {question_title}
Body: {question_body}'''

PROMPT_RETRIEVING_S = '''You will receive a question from a student taking a Tableau course, which includes a title and a body.
The corresponding lecture will also be provided.

Answer the question using only the provided context.

At the end of your response, include the section and lecture names where the context was drawn from, formatted as follows:
Resources:
Section: *Section Title*, Lecture: *Lecture Title*
...
Replace *Section Title* and *Lecture Title* with the appropriate titles.'''

PROMPT_TEMPLATE_RETRIEVING_H = '''This is the question:
{question}

This is the context:
{context}'''

In [ ]:
prompt_creating_question=PromptTemplate.from_template(PROMPT_CREATING_QUESTION)
prompt_retrieving_s=SystemMessagePromptTemplate.from_template(PROMPT_RETRIEVING_S)
prompt_template_retrieving_h=HumanMessagePromptTemplate.from_template(PROMPT_TEMPLATE_RETRIEVING_H)

In [ ]:
prompt_template_retrieving_h

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='This is the question:\n{question}\n\nThis is the context:\n{context}'), additional_kwargs={})

In [ ]:
prompt_creating_question

PromptTemplate(input_variables=['question_body', 'question_lecture', 'question_title'], input_types={}, partial_variables={}, template='Lecture: {question_lecture}\nTitle: {question_title}\nBody: {question_body}')

In [ ]:
chat_propt_template_retrieving=ChatPromptTemplate.from_messages([prompt_retrieving_s,prompt_template_retrieving_h])

In [ ]:
chat_propt_template_retrieving

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You will receive a question from a student taking a Tableau course, which includes a title and a body. \nThe corresponding lecture will also be provided.\n\nAnswer the question using only the provided context.\n\nAt the end of your response, include the section and lecture names where the context was drawn from, formatted as follows: \nResources: \nSection: *Section Title*, Lecture: *Lecture Title* \n...\nReplace *Section Title* and *Lecture Title* with the appropriate titles.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='This is the question:\n{question}\n\nThis is the context:\n{context}'), additional_kwargs={})])

The Q&A chatbot

In [ ]:
test=prompt_creating_question.invoke({"question_lecture": "Adding a custom calculation",
                                  "question_title": "Why are we using SUM here? It's unclear to me.",
                                  "question_body": "This question refers to calculating the GM%."})

In [ ]:
from langchain_core.runnables import chain
@chain
def extract_input(value):
  return {"question":value.text,"context":retriever}


In [ ]:
chain_retrieving=prompt_creating_question|extract_input|chat_propt_template_retrieving|chat|str_output_parser

In [ ]:
result = chain_retrieving.invoke({"question_lecture": "Adding a custom calculation",
                                  "question_title": "Why are we using SUM here? It's unclear to me.",
                                  "question_body": "This question refers to calculating the GM%."})

In [ ]:
chain_test=prompt_creating_question|chat_propt_template_retrieving|chat|str_output_parser

In [ ]:
print(result)

The `SUM()` function is used to aggregate the values of the `Sales` and `Cost` fields. This is necessary because Tableau operates on aggregated data. When you create a calculated field like `GM%`, you are defining a formula that will be applied to each row of your data. However, to display a single value for `GM%` on a worksheet, Tableau needs to know how to combine the individual `Sales` and `Cost` values for all the rows that contribute to that display. `SUM()` tells Tableau to add up all the `Sales` and `Cost` values before performing the division to calculate the `GM%`.

Resources:
Section: Calculations, Section: Calculations
Lecture: Adding a custom calculation


In [ ]:
#chain_test=prompt_creating_question|chat_propt_template_retrieving|chat

In [ ]:
result_Streaming=chain_retrieving.stream({"question_lecture": "Adding a custom calculation",
                                  "question_title": "Why are we using SUM here? It's unclear to me.",
                                  "question_body": "This question refers to calculating the GM%."})

In [ ]:
for chunk in result_Streaming:
  print(chunk,end="")

The `SUM()` function is used to aggregate the values of the `Sales` and `Cost` fields. This is necessary because Tableau operates on aggregated data. When you create a calculated field like `GM%`, you are defining a formula that will be applied to each row of your data. However, to display a single value for `GM%` on a worksheet, Tableau needs to know how to combine the individual `Sales` and `Cost` values for all the rows that contribute to that display. `SUM()` tells Tableau to add up all the `Sales` and `Cost` values before performing the division to calculate the `GM%`.

Resources:
Section: Calculations, Section: Calculations
Lecture: Adding a custom calculation